In [1]:
from sqlalchemy import select
from autotind.db import Person, DB, PType, ImageStatus
import matplotlib.pyplot as plt

db = DB()

session = db.create_session()

In [2]:
from PIL import Image

def is_image_file_valid(image_file):
    try:
        image = Image.open(image_file)
        image.close()
        return True
    except:
        return False

peoples = session.query(Person).all()


def validate_person(person: Person):
    if person.type == PType.REC.value:
        return False
    for photo in person.photos:
        if photo.save_path.exists() and is_image_file_valid(photo.save_path):
            continue
        elif photo.save_path.exists() and photo.save_path.suffix == '.mp4':
            print(photo.save_path.suffix)
            continue
        else:
            return False
    return True


In [3]:
import pandas as pd

full_dataset = []

def get_type(type):
    if type == PType.MATCH.value:
        return 'like'
    return type

for people in peoples:
    for photo in people.photos:
        if photo.save_path.exists() and is_image_file_valid(photo.save_path):
            full_dataset.append([str(photo.save_path), get_type(people.type)])

df = pd.DataFrame(full_dataset, columns=['image', 'type'])

In [4]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2)


In [5]:
print(len(peoples))

820


In [11]:
import torch
torch.cuda.is_available()

True

In [6]:
import flash
from flash.image import ImageClassificationData, ImageClassifier

model = ImageClassifier(backbone="resnet50", labels=["like", "dislike"])
model.cuda()
data = ImageClassificationData.from_data_frame('image', 'type', train_data_frame=train_df, val_data_frame=test_df, batch_size=16)

trainer = flash.Trainer(max_epochs=20, accelerator="gpu")
trainer.finetune(model, data)

/home/charles/code/autotind/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/charles/code/autotind/.venv/lib/python3.8/site-packages/flash/core/trainer.py:213: UserWarning: The model contains a default finetune callback.
  warnings.warn("The model contains a default finetune callback.", UserWarning)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type           | Params
-------------------------------------------------
0 | train_metrics | ModuleDict     | 0     
1 | val_metrics   | ModuleDict     | 0     
2 | test_metrics  | ModuleDict     | 0     
3 | adapter       | DefaultAdapter | 23.5 M
---------------------------------------------

Sanity Checking: 0it [00:00, ?it/s]

/home/charles/code/autotind/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

../aten/src/ATen/native/cuda/Loss.cu:257: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [12,0,0] Assertion `t >= 0 && t < n_classes` failed.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [34]:
with open('dislike_id.csv', 'r') as fd:
    id = fd.readline()
    session.query(Person).filter(Person.id == id).update({ Person.type: PType.DISLIKE.value })
    session.commit()

